# Minimal Example for Python
This notebook contains minimally reproducible example for Python.

In [2]:
import os
import yaml
# print(os.getcwd()) # Check the working directory
# os.chdir(..) # Change directory to main folder and not 'src'
import numpy as np
import pandas as pd
import requests

def load_config(config_file):
    with open(config_file, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc) 

In [8]:
# Load Configuration file
config = load_config('config.yml')
api_url = config['api_url']

# Check the API URL
print(f'API URL: {api_url}')

API URL: http://globalrankings.io/player_ranks


In [11]:
## Specify Query Parameters

# Specify the variable weights
var_weights = {'crosses_str':2, 'interceptions_reg':1, 'shot_assists_reg':3, 'recoveries_to_fouls_rat': 1}

# Specify relevant positions
rel_positions = ['lb5', 'rb5', 'lb', 'rb', 'lwb', 'rwb']

# Analysis Window
window_start = '2017-08-01'
window_end = '2018-06-30'

# Competitions included
comps = ['Domestic league', 'International cup']

# Time Decay
alpha = 0.99

# Minimum number of games
min_games = 10

# Minimum percentage in relevant positions
min_pct = 75

# Minimum minutes tagged
min_tag = 10

In [12]:
# Turn variable weights into string
var_param = '{'+",".join('"'+k+'"'+':'+str(v) for k,v in var_weights.items())+'}'

# Turn relevant positions into string
pos_param = ','.join(rel_positions)

# Turn competitions into string
comps_param = ','.join(comps)

In [13]:
# Send the HTTP Request to API and get output
payload = {'varWeights':var_param, 'positions':pos_param, 'window_start':window_start, 'window_end':window_end, \
    'competitions_included':comps_param, 'time_decay_coef':alpha, 'min_num_games':min_games, 'min_pct_played':min_pct, 'min_tag_threshold':min_tag}
resp = requests.get(api_url, params=payload)
json_out = resp.json()

In [20]:
# Print keys
list(json_out.keys())

['variables',
 'configuration',
 'ingame_statistics',
 'scale_coefficients',
 'player_averages',
 'features',
 'player_ranks']

In [35]:
ranks = json_out['player_ranks']
ranks_df = pd.read_json(ranks).sort_values('player_rating', ascending = False).reset_index()
ranks_df = ranks_df.rename(columns={'index': 'playerId'})
ranks_df.head()

,playerId,crosses_str,interceptions_reg,recoveries_to_fouls_rat,shot_assists_reg,player_rating,target_position,player_rank
0,254,0.917790,0.856513,0.739445,0.882792,0.866692,lb5,1
1,18669,0.792178,0.941096,0.868196,0.850670,0.848214,lb5,2
2,6513,0.833134,0.827287,0.642591,0.920704,0.837184,lb5,3
3,131184,0.724783,0.935271,0.850670,0.771680,0.790110,lb5,4
4,3529,0.713051,0.909045,0.613207,0.871115,0.787318,lb5,5


In [29]:
player_averages = json_out['player_averages']
player_averages = pd.read_json(player_averages)
player_averages.head()

,playerId,minutesOnField,minutesTagged,goals,assists,shots,headShots,yellowCards,redCards,directRedCards,...,combined_shots_on_target_scaled,inplay_shots_scaled,inplay_shots_on_target_scaled,freekick_shots_scaled,freekick_shots_on_target_scaled,penalty_shots_scaled,penalty_shots_on_target_scaled,non_penalty_shots_scaled,non_penalty_shots_on_target_scaled,num_games
0,107,93.800000,93.800000,0.000000,0.000000,0.097519,0.000000,0.161112,0.0,0.0,...,0.009003,0.027775,0.009003,0.000000,0.000000,0.0,0.0,0.009003,0.009003,29.521734
1,110,78.454545,78.454545,0.000000,0.092167,1.026950,0.047780,0.194557,0.0,0.0,...,0.100715,0.201881,0.082560,0.038337,0.018154,0.0,0.0,0.120897,0.100715,23.888512
2,132,89.454545,89.454545,0.000000,0.000000,0.185333,0.000000,0.000000,0.0,0.0,...,0.000000,0.045397,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,11.124705
3,160,87.909091,87.909091,0.152607,0.000000,1.042079,0.130719,0.132350,0.0,0.0,...,0.102802,0.247136,0.102802,0.000000,0.000000,0.0,0.0,0.102802,0.102802,22.764153
4,254,93.314286,93.314286,0.028032,0.000000,0.479299,0.000000,0.280721,0.0,0.0,...,0.036295,0.098860,0.020362,0.025274,0.015933,0.0,0.0,0.045636,0.036295,34.469244
